In [6]:
from scipy.spatial.transform import Rotation as R
import numpy as np
import plotly.graph_objects as go

# Visualize Using Transformation Matrix to Move Points

In [229]:
def draw_simple_transform(fig, homo_trans, points = None):
    """fig = plotly figure object
       homo_trans = homogenous transformation matrix (4x4)
       points = points coordinates in origonal frame"""

    x0 = homo_trans[0,3]
    y0 = homo_trans[1,3]
    z0 = homo_trans[2,3]
#     x1, y1, z1 =  np.eye(3) @ np.linalg.pinv(homo_trans[:3,:3])
    x1, y1, z1 =  np.eye(3) @ homo_trans[:3,:3]
    x1 += homo_trans[0,-1]
    y1 += homo_trans[1,-1]
    z1 += homo_trans[2,-1]
    #old axis
    fig.add_trace(go.Scatter3d(x=[0, 1], y=[0, 0], z=[0, 0], mode='lines', line=dict(color='rgba(255, 0, 0, 0.2)', width=5)))
    fig.add_trace(go.Scatter3d(x=[0, 0], y=[0, 1], z=[0, 0], mode='lines', line=dict(color='rgba(0, 255, 0, 0.2)', width=5)))
    fig.add_trace(go.Scatter3d(x=[0, 0], y=[0, 0], z=[0, 1], mode='lines', line=dict(color='rgba(0, 0, 255, 0.2)', width=5)))
    #new x
    fig.add_trace(go.Scatter3d(x=[x0, x1[0]], y=[y0, y1[0]], z=[z0, z1[0]], mode='lines', line=dict(color='red', width=5)))
    fig.add_trace(go.Scatter3d(x=[x0, x1[1]], y=[y0, y1[1]], z=[z0, z1[1]], mode='lines', line=dict(color='green', width=5)))
    fig.add_trace(go.Scatter3d(x=[x0, x1[2]], y=[y0, y1[2]], z=[z0, z1[2]], mode='lines', line=dict(color='blue', width=5)))

    if points is not None:
        #apply matrix multiplication
        new_points = np.append(points, np.ones([len(points),1]), axis = 1) @ homo_trans.T
        
        #draw old points
        fig.add_trace(go.Scatter3d(
            x=points[:,0],
            y=points[:,1],
            z=points[:,2],
            mode='markers',
            marker=dict(
                size=5,          # Size of the points
                color='rgba(32, 32, 32, 0.4)',       # Color of the points
                opacity=0.8       # Opacity of the points
            )
        ))
        #draw new points
        fig.add_trace(go.Scatter3d(
            x=new_points[:,0],
            y=new_points[:,1],
            z=new_points[:,2],
            mode='markers',
            marker=dict(
                size=5,          # Size of the point
                color='rgba(32, 32, 32, 1.)',       # Color of the point
                opacity=0.8       # Opacity of the point
            )
        ))
        return new_points
    

In [230]:
# Create a 3D figure
fig = go.Figure()

#initialize 3x3 rotation matrix
euler_angles = [0.,-0.35,0.5] #rotation about xaxis, yaxis, zaxis
rotm = R.from_euler('xyz', euler_angles).as_matrix()
print("\n rotation matrix (3x3) \n", rotm, "\n")

#initialize translation vector
# trans_vec = np.array([[0., 0., 0]]) #[x, y, z]
trans_vec = np.array([[0.2, 1., 1.5]]) #[x, y, z]
print("\n translation vector (3x1) \n", trans_vec)

#resize rotation matrix to 4x4 so it can be used with homogenous cooridnates
homo_trans = np.append(np.append(rotm, np.zeros([3,1]), axis = 1), np.array([[0,0,0,1]]), axis = 0)
#update homo_trans to include translation as well
homo_trans[:3,-1] = trans_vec
print("\n homogenous transformation matrix (4x4) \n", homo_trans ,"\n")

#initialize test points
test_points = np.array([[2,2,3], [2,2,2], [3,3,3], [3,3,2]])

#run our function
new_point_locations = draw_simple_transform(fig, homo_trans, points = test_points)
fig.show()

print("test point before:", test_points[0])
print("test point after transformation:", new_point_locations[0,:3])


 rotation matrix (3x3) 
 [[ 0.82437711 -0.47942554 -0.30092114]
 [ 0.45035927  0.87758256 -0.16439397]
 [ 0.34289781  0.          0.93937271]] 


 translation vector (3x1) 
 [[0.2 1.  1.5]]

 homogenous transformation matrix (4x4) 
 [[ 0.82437711 -0.47942554 -0.30092114  0.2       ]
 [ 0.45035927  0.87758256 -0.16439397  1.        ]
 [ 0.34289781  0.          0.93937271  1.5       ]
 [ 0.          0.          0.          1.        ]] 



test point before: [2 2 3]
test point after transformation: [-0.01286026  3.16270176  5.00391375]


# Visualize Subsequent Transformations

In [258]:
def draw_frame(fig, homo_trans, alpha = 1.):
    """fig = plotly figure object
       homo_trans = homogenous transformation matrix (4x4)"""

    x0 = homo_trans[0,3]
    y0 = homo_trans[1,3]
    z0 = homo_trans[2,3]
    x1, y1, z1 =  np.eye(3) @ homo_trans[:3,:3]
    x1 += homo_trans[0,-1]
    y1 += homo_trans[1,-1]
    z1 += homo_trans[2,-1]
    fig.add_trace(go.Scatter3d(x=[x0, x1[0]], y=[y0, y1[0]], z=[z0, z1[0]], mode='lines', line=dict(color='rgba(255, 0, 0, ' +str(alpha) + ')', width=5)))
    fig.add_trace(go.Scatter3d(x=[x0, x1[1]], y=[y0, y1[1]], z=[z0, z1[1]], mode='lines', line=dict(color='rgba(0, 255, 0, ' +str(alpha) + ')', width=5)))
    fig.add_trace(go.Scatter3d(x=[x0, x1[2]], y=[y0, y1[2]], z=[z0, z1[2]], mode='lines', line=dict(color='rgba(0, 0, 255, ' +str(alpha) + ')', width=5)))

In [261]:
# Create a 3D figure
fig = go.Figure()
fig.update_layout(
    scene=dict(
        aspectmode='data'  # ensures all axes are scaled equally
    )
)

#initialize 3x3 rotation matrix
euler_angles = [0.,0.,0.] #rotation about xaxis, yaxis, zaxis
# print("\n rotation matrix (3x3) \n", rotm, "\n")

#initialize translation vector
trans_vec = np.array([[0., 0., 0]]) #[x, y, z]
# print("\n translation vector (3x1) \n", trans_vec)

#resize rotation matrix to 4x4 so it can be used with homogenous cooridnates
#update homo_trans to include translation as well
print("\n homogenous transformation matrix (4x4) for pose 0 \n", homo_trans ,"\n")


runLen = 5

draw_frame(fig, np.eye(4), alpha = 1/runLen)
for i in range(runLen):
    #update translation
    trans_vec += np.array([1.5,np.sin(i/10),.1])
    
    #update rotation
    euler_angles += np.array([0, -0.05, 0.2])
    rotm = R.from_euler('xyz', euler_angles).as_matrix()
    
    #update homo_trans
    homo_trans = np.append(np.append(rotm, np.zeros([3,1]), axis = 1), np.array([[0,0,0,1]]), axis = 0)
    homo_trans[:3,-1] = trans_vec
    print("\n homogenous transformation matrix (4x4) for pose", i + 1, "\n", homo_trans ,"\n")
    
    #add to figure
    draw_frame(fig, homo_trans, (i+1)/(runLen+1))

fig.show()


 homogenous transformation matrix (4x4) for pose 0 
 [[ 0.52350562 -0.84147098 -0.13367293  7.5       ]
 [ 0.81531169  0.54030231 -0.20818325  0.9834413 ]
 [ 0.24740396  0.          0.96891242  0.5       ]
 [ 0.          0.          0.          1.        ]] 


 homogenous transformation matrix (4x4) for pose 1 
 [[ 0.97884175 -0.19866933 -0.04898291  1.5       ]
 [ 0.19842105  0.98006658 -0.00992933  0.        ]
 [ 0.04997917  0.          0.99875026  0.1       ]
 [ 0.          0.          0.          1.        ]] 


 homogenous transformation matrix (4x4) for pose 2 
 [[ 9.16459526e-01 -3.89418342e-01 -9.19526660e-02  3.00000000e+00]
 [ 3.87472873e-01  9.21060994e-01 -3.88769636e-02  9.98334166e-02]
 [ 9.98334166e-02  3.46944695e-18  9.95004165e-01  2.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 


 homogenous transformation matrix (4x4) for pose 3 
 [[ 0.81606799 -0.56464247 -0.12333661  4.5       ]
 [ 0.55830215  0.82533561 -0.08437912  0.2985027